In [1]:
!python -V

Python 3.11.5


In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [40]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.utils import resample
from scipy.sparse import csr_matrix

In [21]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [7]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('file:///C:/Users/Cokroaminoto/Documents/Analisis Data/Data '
 'Scientist/DataTalksClub/MLOps - zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1727228916584, experiment_id='1', last_update_time=1727228916584, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
#df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
#df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

df_train = read_dataframe('/datasets/dtc/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/datasets/dtc/green_tripdata_2021-02.parquet')

In [10]:
df_train.shape

(73908, 21)

In [11]:
df_val.shape

(61921, 21)

In [12]:
# Take 10% of the data randomly
df_train = df_train.sample(frac=0.1, random_state=42)
df_val = df_val.sample(frac=0.1, random_state=42)

In [13]:
df_train.shape

(7391, 21)

In [14]:
df_val.shape

(6192, 21)

In [15]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [16]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.486348156352333

In [19]:
# Create directories if they don't exist
try:
    os.makedirs('models/', exist_ok=True)
except Exception as e:
    print(f"Error creating models folder: {e}")

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [20]:
with mlflow.start_run():

    mlflow.set_tag("developer", "adi")

    mlflow.log_param("train-data-path", "/datasets/dtc/green_tripdata_2021-01.parquet")
    #mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("sample-size-train-data", df_train.shape[0])  # Record the number of samples taken
    mlflow.log_param("valid-data-path", "/datasets/dtc/green_tripdata_2021-02.parquet")
    #mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    mlflow.log_param("sample-size-valid-data", df_valid.shape[0])  # Record the number of samples taken

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            #num_boost_round=1000,  # for processor multi-core Intel i5 with 8GB RAM or higher
            num_boost_round=200,  # Reduced from 1000 for processor Intel Celeron N3350 with 4GB RAM 
            evals=[(valid, 'validation')],
            #early_stopping_rounds=50  # for processor multi-core Intel i5 with 8GB RAM or higher 
            early_stopping_rounds=20  # Reduced from 50 for processor Intel Celeron N3350 with 4GB RAM 
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}


In [24]:
search_space = {
    #'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # for processor multi-core Intel i5 with 8GB RAM or higher
    'max_depth': scope.int(hp.quniform('max_depth', 4, 20, 1)),  # Reduced range for processor Intel Celeron N3350 with 4GB RAM 
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    #'objective': 'reg:linear', # older version
    'objective': 'reg:squarederror',  # newer version
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:39:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.04451                                                                                            
[1]	validation-rmse:6.91873                                                                                            
[2]	validation-rmse:6.91370                                                                                            
[3]	validation-rmse:6.91251                                                                                            
[4]	validation-rmse:6.90556                                                                                            
[5]	validation-rmse:6.89537                                                                                            
[6]	validation-rmse:6.89635                                                                                            
[7]	validation-rmse:6.89262                                                                                            
[8]	validation-rmse:6.89484             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



  2%|█                                                 | 1/50 [00:12<10:03, 12.33s/trial, best loss: 6.876838310257262]

...

[0]	validation-rmse:11.41251                                                                                           
 98%|█████████████████████████████████████████████████ | 49/50 [14:45<00:22, 22.51s/trial, best loss: 6.78515925152334]

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:54:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:10.58456                                                                                           
[2]	validation-rmse:9.89506                                                                                            
[3]	validation-rmse:9.32495                                                                                            
[4]	validation-rmse:8.85441                                                                                            
[5]	validation-rmse:8.47122                                                                                            
[6]	validation-rmse:8.15697                                                                                            
[7]	validation-rmse:7.90285                                                                                            
[8]	validation-rmse:7.69935                                                                                            
[9]	validation-rmse:7.53510             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|██████████████████████████████████████████████████| 50/50 [14:58<00:00, 17.97s/trial, best loss: 6.78515925152334]


In [26]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
def objective(params):
    #mlflow.end_run()  # End the previous run if it exists
    with mlflow.start_run(nested=True): # Use Nested Runs
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=200,
            evals=[(valid, 'validation')],
            early_stopping_rounds=20
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}


In [29]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    #best_params = {
    #    'learning_rate': 0.09585355369315604,
    #    'max_depth': 30,
    #    'min_child_weight': 1.060597050922164,
    #    'objective': 'reg:linear',
    #    'reg_alpha': 0.018060244040060163,
    #    'reg_lambda': 0.011658731377413597,
    #    'seed': 42
    #}
    # Using best_result to set best_params
    best_params = {
        'learning_rate': best_result['learning_rate'],
        'max_depth': int(best_result['max_depth']),  # Make sure to convert to int
        'min_child_weight': best_result['min_child_weight'],
        #'objective': 'reg:linear', #older version
        'objective': 'reg:squarederror',  
        'reg_alpha': best_result['reg_alpha'],
        'reg_lambda': best_result['reg_lambda'],
        'seed': 42
    }

    best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=200, #1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=20 #50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:09:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.13259                                                                                            
[1]	validation-rmse:6.91844                                                                                            
[2]	validation-rmse:6.90432                                                                                            
[3]	validation-rmse:6.89006                                                                                            
[4]	validation-rmse:6.87571                                                                                            
[5]	validation-rmse:6.87444                                                                                            
[6]	validation-rmse:6.87527                                                                                            
[7]	validation-rmse:6.87217                                                                                            
[8]	validation-rmse:6.87226             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



  2%|█                                                 | 1/50 [00:09<07:39,  9.39s/trial, best loss: 6.864888798804896]

...

[1]	validation-rmse:10.09677                                                                                           
[2]	validation-rmse:9.30952                                                                                            
[3]	validation-rmse:8.70761                                                                                            
[4]	validation-rmse:8.24547                                                                                            
[5]	validation-rmse:7.90087                                                                                            
[6]	validation-rmse:7.64235                                                                                            
[7]	validation-rmse:7.45129                                                                                            
[8]	validation-rmse:7.29972                                                                                            
[9]	validation-rmse:7.19018             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|█████████████████████████████████████████████████| 50/50 [09:45<00:00, 11.71s/trial, best loss: 6.790096051333509]
[0]	validation-rmse:9.99779
[1]	validation-rmse:8.58895
[2]	validation-rmse:7.80349
[3]	validation-rmse:7.37479
[4]	validation-rmse:7.14621
[5]	validation-rmse:7.01258
[6]	validation-rmse:6.94435
[7]	validation-rmse:6.90516
[8]	validation-rmse:6.87912
[9]	validation-rmse:6.86029
[10]	validation-rmse:6.85335
[11]	validation-rmse:6.84362
[12]	validation-rmse:6.84287
[13]	validation-rmse:6.84243
[14]	validation-rmse:6.84169
[15]	validation-rmse:6.83336
[16]	validation-rmse:6.83549
[17]	validation-rmse:6.83595
[18]	validation-rmse:6.83437
[19]	validation-rmse:6.83373
[20]	validation-rmse:6.83162
[21]	validation-rmse:6.82980
[22]	validation-rmse:6.82709
[23]	validation-rmse:6.82711
[24]	validation-rmse:6.82633
[25]	validation-rmse:6.82577
[26]	validation-rmse:6.82432
[27]	validation-rmse:6.82308
[28]	validation-rmse:6.82216
[29]	validation-rmse:6.81941
[30]	validation-rmse

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/09/26 15:21:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


'\nwith mlflow.start_run():\n    best_result[\'max_depth\'] = int(best_result[\'max_depth\'])  # Ensure max_depth is an integer\n    mlflow.log_params(best_result)\n    booster = xgb.train(\n        params=best_result,\n        dtrain=train,\n        #num_boost_round=1000, # for processor multi-core Intel i5 with 8GB RAM or higher\n        num_boost_round=200,  # Reduced from 1000 for processor Intel Celeron N3350 with 4GB RAM \n        evals=[(valid, \'validation\')],\n        #early_stopping_rounds=50 # for processor multi-core Intel i5 with 8GB RAM or higher\n        early_stopping_rounds=20  # Reduced from 50 for processor Intel Celeron N3350 with 4GB RAM \n    )\n    y_pred = booster.predict(valid)\n    rmse = mean_squared_error(y_val, y_pred, squared=False)\n    mlflow.log_metric("rmse", rmse)\n\n    with open("models/preprocessor.b", "wb") as f_out:\n        pickle.dump(dv, f_out)\n    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")\n\n    mlflow.xgboo

In [46]:
#mlflow.sklearn.autolog(log_input_examples=False)  # Disable logging of input examples
mlflow.sklearn.autolog() # Enable automatic logging

# Print shapes for debugging
print("Shape of X_train before any transformation:", X_train.shape)
print("Shape of y_train before any transformation:", y_train.shape)

# If X_train and X_val are sparse matrices, convert them to dense
if isinstance(X_train, csr_matrix):
    X_train_dense = X_train.toarray()
else:
    X_train_dense = X_train

if isinstance(X_val, csr_matrix):
    X_val_dense = X_val.toarray()
else:
    X_val_dense = X_val

# Check shapes after conversion
print("Shape of X_train_dense after conversion:", X_train_dense.shape)
print("Shape of y_train after conversion:", y_train.shape)

# Ensure X_train_dense and y_train have consistent lengths
assert X_train_dense.shape[0] == y_train.shape[0], "X_train and y_train must have the same number of samples."

# Define sample size (10% of original data)
sample_size = int(0.1 * X_train_dense.shape[0])

# Randomly sample 10% of the training set
X_train_sampled, y_train_sampled = resample(X_train_dense, y_train, n_samples=sample_size, random_state=42)

# Loop through different models and apply optimizations
for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
    with mlflow.start_run():
        
        # Log parameters and artifacts
        mlflow.log_param("train-data-path", "/datasets/dtc/green_tripdata_2021-01.parquet")
        #mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
        mlflow.log_param("sample-size-train-data", df_train.shape[0])  # Record the number of samples taken
        mlflow.log_param("valid-data-path", "/datasets/dtc/green_tripdata_2021-02.parquet")
        #mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
        mlflow.log_param("sample-size-valid-data", df_valid.shape[0])  # Record the number of samples taken
        
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        # Configure the model based on its class
        if model_class == LinearSVR:
            # Set parameters for LinearSVR
            mlmodel = model_class(max_iter=1000, dual=True, tol=1e-2)  # Use dual=True for compatibility
        elif model_class in (RandomForestRegressor, ExtraTreesRegressor):
            # Use parallel processing for tree-based models
            mlmodel = model_class(n_jobs=-1)
        elif model_class == GradientBoostingRegressor:
            # Enable early stopping for Gradient Boosting
            mlmodel = model_class(n_iter_no_change=5, validation_fraction=0.1)
        else:
            mlmodel = model_class()

        # Fit the model using the sampled training data
        mlmodel.fit(X_train_sampled, y_train_sampled)

        # Make predictions using the original validation set
        y_pred = mlmodel.predict(X_val_dense)

        # Log RMSE using the original validation set
        rmse = root_mean_squared_error(y_val, y_pred)
        #rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        # Print training time
        print(f"Finished training {model_class.__name__} with RMSE: {rmse}")


Shape of X_train before any transformation: (7391, 3418)
Shape of y_train before any transformation: (7391,)
Shape of X_train_dense after conversion: (7391, 3418)
Shape of y_train after conversion: (7391,)
Finished training RandomForestRegressor with RMSE: 7.259841238023631
Finished training GradientBoostingRegressor with RMSE: 7.020166648537414
Finished training ExtraTreesRegressor with RMSE: 7.403640765063365
Finished training LinearSVR with RMSE: 1506.4530306035401
